# Analyzing Portfolio Risk and Return

In [1]:
# Initial imports
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import numpy as np
import hvplot.pandas
import plotly.express as px


%matplotlib inline



### Import the Data


In [2]:
# Load .env enviroment variables
load_dotenv()

True

In [3]:
alpaca_api_key = os.getenv("APCA_API_KEY_ID")
alpaca_secret_key = os.getenv("APCA_SECRET_KEY")

In [4]:
# Set current amount of shares
#Tech

my_amzn = 100
my_nflx = 100
my_spy = 100
my_goog = 100
my_fab = 100

# ETFs
my_voo = 100
my_vgt = 100


In [5]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [6]:
# Formatting current date as ISO format
start_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-09-07", tz="America/New_York").isoformat()
# Setting the tickers
tickers = ["FB", "APPL","AMZN","NFLX","GOOG","SPY"]

# Setting timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Getting current closing prices for the tickers
df_tech_stock_data = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Reorganize the DataFrame
# Separate ticker data
FB = df_tech_stock_data[df_tech_stock_data['symbol']=='FB'].drop('symbol', axis=1)
APPL = df_tech_stock_data[df_tech_stock_data['symbol']=='APPL'].drop('symbol', axis=1)
SPY = df_tech_stock_data[df_tech_stock_data['symbol']=='SPY'].drop('symbol', axis=1)
AMZN = df_tech_stock_data[df_tech_stock_data['symbol']=='AMZN'].drop('symbol', axis=1)
NFLX = df_tech_stock_data[df_tech_stock_data['symbol']=='NFLX'].drop('symbol', axis=1)
GOOG = df_tech_stock_data[df_tech_stock_data['symbol']=='GOOG'].drop('symbol', axis=1)


# Concatenate the ticker DataFrames
df_tech_stock_data = pd.concat([GOOG,APPL,SPY,AMZN,NFLX,FB],axis=1, keys=['GOOG','APPL','SPY','AMZN','NFLX','FB'])

# Preview DataFrame
df_tech_stock_data.head()
df_tech_stock_data.tail()

GOOG                                      \
                              open     high      low   close    volume   
timestamp                                                                
2022-08-31 04:00:00+00:00  111.630  111.770  109.050  109.15  25884888   
2022-09-01 04:00:00+00:00  109.200  111.220  108.190  110.55  22783719   
2022-09-02 04:00:00+00:00  111.340  111.675  108.130  108.68  20681559   
2022-09-06 04:00:00+00:00  108.135  108.880  106.510  107.48  20557394   
2022-09-07 04:00:00+00:00  107.760  110.990  107.615  110.48  23168179   

                                                  APPL           ...     NFLX  \
                          trade_count        vwap open high low  ...   volume   
timestamp                                                        ...            
2022-08-31 04:00:00+00:00      225813  110.003290  NaN  NaN NaN  ...  8615748   
2022-09-01 04:00:00+00:00      278560  109.577315  NaN  NaN NaN  ...  7292904   
2022-09-02 04:00:00+00:00      229808  109.749947  NaN  NaN NaN  ...  8163451   
2022-09-06 04:00:00+00:00      246791  107.564535  NaN  NaN NaN  ...  7465692   
2022-09-07 04:00:00+00:00      248533  109.595898  NaN  NaN NaN  ...  8147848   

                                                    FB                        \
                          trade_count        vwap open high low close volume   
timestamp                                                                      
2022-08-31 04:00:00+00:00      126819  227.669035  NaN  NaN NaN   NaN    NaN   
2022-09-01 04:00:00+00:00      123511  224.822034  NaN  NaN NaN   NaN    NaN   
2022-09-02 04:00:00+00:00      128928  229.017513  NaN  NaN NaN   NaN    NaN   
2022-09-06 04:00:00+00:00      129774  218.485956  NaN  NaN NaN   NaN    NaN   
2022-09-07 04:00:00+00:00      115048  224.690090  NaN  NaN NaN   NaN    NaN   

                                            
                          trade_count vwap  
timestamp                                   
2022-08-31 04:00:00+00:00         NaN  NaN  
2022-09-01 04:00:00+00:00         NaN  NaN  
2022-09-02 04:00:00+00:00         NaN  NaN  
2022-09-06 04:00:00+00:00         NaN  NaN  
2022-09-07 04:00:00+00:00         NaN  NaN  

[5 rows x 42 columns]

#explian how i dropped apple stock and cleaned facebook

## ETFs
```
Vanguard S&P 500 ETF (VOO)
Vanguard Information Technology ETF (VGT)
```

In [7]:
# Format current date as ISO format
start_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-08-07", tz="America/New_York").isoformat()
# Set the tickers
tickers = ["VOO", "VGT",'SPY']

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for VOO and VGT
df_etf = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Reorganize the DataFrame
# Separate ticker data
VOO = df_etf[df_etf['symbol']=='VOO'].drop('symbol', axis=1)
VGT = df_etf[df_etf['symbol']=='VGT'].drop('symbol', axis=1)
SPY = df_etf[df_etf['symbol']=='SPY'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames
df_etf = pd.concat([VOO,VGT,SPY],axis=1, keys=['VOO','VGT','SPY'])

# Preview DataFrame
df_etf.head()
df_etf.tail()

VOO                                    \
                             open     high     low   close   volume   
timestamp                                                             
2022-08-01 04:00:00+00:00  376.12  380.010  375.44  377.65  4374908   
2022-08-02 04:00:00+00:00  376.11  379.627  373.95  375.21  4054107   
2022-08-03 04:00:00+00:00  377.19  382.100  376.91  381.07  3826838   
2022-08-04 04:00:00+00:00  380.88  381.530  379.19  380.77  3274628   
2022-08-05 04:00:00+00:00  376.60  380.700  376.54  379.98  2843831   

                                                      VGT                   \
                          trade_count        vwap    open    high      low   
timestamp                                                                    
2022-08-01 04:00:00+00:00       88166  377.840171  367.20  373.30  365.640   
2022-08-02 04:00:00+00:00       93254  376.597738  366.11  371.93  364.530   
2022-08-03 04:00:00+00:00       83236  380.034817  369.53  379.07  369.530   
2022-08-04 04:00:00+00:00       74004  380.565442  377.74  379.53  374.941   
2022-08-05 04:00:00+00:00       82991  378.877859  373.35  380.12  373.060   

                           ...                                     SPY  \
                           ...  volume trade_count        vwap    open   
timestamp                  ...                                           
2022-08-01 04:00:00+00:00  ...  572617       23880  370.083886  409.15   
2022-08-02 04:00:00+00:00  ...  547046       26805  368.244635  409.12   
2022-08-03 04:00:00+00:00  ...  482865       14945  376.632144  410.30   
2022-08-04 04:00:00+00:00  ...  324378       11706  377.862618  414.37   
2022-08-05 04:00:00+00:00  ...  338058       14358  377.190347  409.66   

                                                                         \
                             high     low   close    volume trade_count   
timestamp                                                                 
2022-08-01 04:00:00+00:00  413.41  408.40  410.77  69633616      516221   
2022-08-02 04:00:00+00:00  413.00  406.82  408.06  63113622      502751   
2022-08-03 04:00:00+00:00  415.68  410.00  414.45  67573063      463932   
2022-08-04 04:00:00+00:00  415.09  412.44  414.17  45565581      364658   
2022-08-05 04:00:00+00:00  414.15  409.60  413.47  56751151      455450   

                                       
                                 vwap  
timestamp                              
2022-08-01 04:00:00+00:00  411.079113  
2022-08-02 04:00:00+00:00  409.635855  
2022-08-03 04:00:00+00:00  413.267159  
2022-08-04 04:00:00+00:00  414.080458  
2022-08-05 04:00:00+00:00  412.292366  

[5 rows x 21 columns]

### Analyze the Performance

#### Clean And Organize The Data

#### Tech Stocks

In [8]:
# extract close price from tech stock data 
tech_close_price = pd.DataFrame()
for i in [ticker[0] for ticker in df_tech_stock_data]:
    tech_close_price[i] = df_tech_stock_data[i]['close']
print(tech_close_price)
tech_close_price

                              GOOG  APPL     SPY     AMZN    NFLX      FB
timestamp                                                                
2020-08-07 04:00:00+00:00  1494.49   NaN  334.57  3167.46  494.73  268.44
2020-08-10 04:00:00+00:00  1496.10   NaN  335.56  3148.16  483.38  263.00
2020-08-11 04:00:00+00:00  1480.32   NaN  332.82  3080.67  466.93  256.13
2020-08-12 04:00:00+00:00  1506.62   NaN  337.31  3162.24  475.47  259.89
2020-08-13 04:00:00+00:00  1518.45   NaN  336.81  3161.02  481.33  261.30
...                            ...   ...     ...      ...     ...     ...
2022-08-31 04:00:00+00:00   109.15   NaN  395.18   126.77  223.56     NaN
2022-09-01 04:00:00+00:00   110.55   NaN  396.42   127.82  230.04     NaN
2022-09-02 04:00:00+00:00   108.68   NaN  392.24   127.51  226.11     NaN
2022-09-06 04:00:00+00:00   107.48   NaN  390.76   126.11  218.39     NaN
2022-09-07 04:00:00+00:00   110.48   NaN  397.78   129.48  228.96     NaN

[525 rows x 6 columns]


,GOOG,APPL,SPY,AMZN,NFLX,FB
timestamp,,,,,,
2020-08-07 04:00:00+00:00,1494.49,NaN,334.57,3167.46,494.73,268.44
2020-08-10 04:00:00+00:00,1496.10,NaN,335.56,3148.16,483.38,263.00
2020-08-11 04:00:00+00:00,1480.32,NaN,332.82,3080.67,466.93,256.13
2020-08-12 04:00:00+00:00,1506.62,NaN,337.31,3162.24,475.47,259.89
2020-08-13 04:00:00+00:00,1518.45,NaN,336.81,3161.02,481.33,261.30
...,...,...,...,...,...,...
2022-08-31 04:00:00+00:00,109.15,NaN,395.18,126.77,223.56,NaN
2022-09-01 04:00:00+00:00,110.55,NaN,396.42,127.82,230.04,NaN
2022-09-02 04:00:00+00:00,108.68,NaN,392.24,127.51,226.11,NaN


In [9]:
# Drop the APPL column 
new_tech_close = tech_close_price.drop(columns = "APPL")
new_tech_close.dropna()

,GOOG,SPY,AMZN,NFLX,FB
timestamp,,,,,
2020-08-07 04:00:00+00:00,1494.49,334.57,3167.46,494.73,268.44
2020-08-10 04:00:00+00:00,1496.10,335.56,3148.16,483.38,263.00
2020-08-11 04:00:00+00:00,1480.32,332.82,3080.67,466.93,256.13
2020-08-12 04:00:00+00:00,1506.62,337.31,3162.24,475.47,259.89
2020-08-13 04:00:00+00:00,1518.45,336.81,3161.02,481.33,261.30
...,...,...,...,...,...
2022-06-02 04:00:00+00:00,2354.92,417.34,2510.22,205.09,198.86
2022-06-03 04:00:00+00:00,2290.92,410.54,2447.00,198.98,190.78
2022-06-06 04:00:00+00:00,2340.21,411.79,124.79,197.14,194.25


#### ETFs

In [10]:
# loop through the etf close price 
df_etf_close = pd.DataFrame()
for i in [ticker[0] for ticker in df_etf]:
    df_etf_close[i] = df_etf[i]['close']
df_etf_close

,VOO,VGT,SPY
timestamp,,,
2020-08-07 04:00:00+00:00,307.36,303.19,334.57
2020-08-10 04:00:00+00:00,308.34,302.23,335.56
2020-08-11 04:00:00+00:00,305.77,296.88,332.82
2020-08-12 04:00:00+00:00,309.96,303.39,337.31
2020-08-13 04:00:00+00:00,309.51,304.13,336.81
...,...,...,...
2022-08-01 04:00:00+00:00,377.65,369.54,410.77
2022-08-02 04:00:00+00:00,375.21,367.63,408.06
2022-08-03 04:00:00+00:00,381.07,378.29,414.45


In [11]:
# Clean the etf close_price data
new_etf_close = df_etf_close.dropna()
new_etf_close

,VOO,VGT,SPY
timestamp,,,
2020-08-07 04:00:00+00:00,307.36,303.19,334.57
2020-08-10 04:00:00+00:00,308.34,302.23,335.56
2020-08-11 04:00:00+00:00,305.77,296.88,332.82
2020-08-12 04:00:00+00:00,309.96,303.39,337.31
2020-08-13 04:00:00+00:00,309.51,304.13,336.81
...,...,...,...
2022-08-01 04:00:00+00:00,377.65,369.54,410.77
2022-08-02 04:00:00+00:00,375.21,367.63,408.06
2022-08-03 04:00:00+00:00,381.07,378.29,414.45


##### Daily & Cumulative Returns

###### TECH


In [12]:
# Convert the dataframe of NAVs and prices to daily returns
tech_daily_returns = new_tech_close.pct_change().dropna()
tech_daily_returns.head()

,GOOG,SPY,AMZN,NFLX,FB
timestamp,,,,,
2020-08-10 04:00:00+00:00,0.001077,0.002959,-0.006093,-0.022942,-0.020265
2020-08-11 04:00:00+00:00,-0.010547,-0.008165,-0.021438,-0.034031,-0.026122
2020-08-12 04:00:00+00:00,0.017766,0.013491,0.026478,0.018290,0.014680
2020-08-13 04:00:00+00:00,0.007852,-0.001482,-0.000386,0.012325,0.005425
2020-08-14 04:00:00+00:00,-0.007060,0.000119,-0.004113,0.002805,-0.000230


In [13]:
tech_daily_returns.describe()

,GOOG,SPY,AMZN,NFLX,FB
count,524.000000,524.000000,524.000000,524.000000,524.000000
mean,-0.000841,0.000395,-0.001962,-0.000902,-0.000255
std,0.045703,0.011364,0.047524,0.032476,0.025604
min,-0.951267,-0.040312,-0.949003,-0.351166,-0.263901
25%,-0.008763,-0.005480,-0.012220,-0.014059,-0.011214
50%,0.001241,0.000848,0.000145,0.000233,0.000000
75%,0.011044,0.007564,0.011684,0.013148,0.012890
max,0.077390,0.031794,0.135359,0.168543,0.176138


In [14]:
# Cumulative returns of the tech portfolios and the S&P 500
tech_cumulative_returns = (1 + tech_daily_returns).cumprod()
tech_cumulative_returns.head()


,GOOG,SPY,AMZN,NFLX,FB
timestamp,,,,,
2020-08-10 04:00:00+00:00,1.001077,1.002959,0.993907,0.977058,0.979735
2020-08-11 04:00:00+00:00,0.990519,0.994769,0.972599,0.943808,0.954142
2020-08-12 04:00:00+00:00,1.008116,1.008190,0.998352,0.961070,0.968149
2020-08-13 04:00:00+00:00,1.016032,1.006695,0.997967,0.972915,0.973402
2020-08-14 04:00:00+00:00,1.008859,1.006815,0.993863,0.975643,0.973178


In [15]:
# Plot cumulative returns 
tech_cumulative_returns.hvplot(kind = "line", title="Tech Funds portfolios and the S&P 500 Cumulative Returns")

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

###### ETFs

In [16]:
# ETF daily returns
etf_daily_returns = new_etf_close.pct_change().dropna()
etf_daily_returns.head()

,VOO,VGT,SPY
timestamp,,,
2020-08-10 04:00:00+00:00,0.003188,-0.003166,0.002959
2020-08-11 04:00:00+00:00,-0.008335,-0.017702,-0.008165
2020-08-12 04:00:00+00:00,0.013703,0.021928,0.013491
2020-08-13 04:00:00+00:00,-0.001452,0.002439,-0.001482
2020-08-14 04:00:00+00:00,-0.000032,-0.002466,0.000119


In [17]:
# ETF cumulative returns
etf_cumulative_returns = (1 + etf_daily_returns).cumprod()

In [18]:
# Plot ETF cumulative returns
etf_cumulative_returns.hvplot(kind = "line", title="ETFs Funds portfolios and the S&P 500 Cumulative Returns")

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

#### Plotly vs hvplot

In [19]:
etf_returns_hvplot = etf_daily_returns.hvplot(kind='line', title='ETFs Performance')

In [20]:
etf_daily_returns = etf_daily_returns.reset_index()
etf_daily_returns = etf_daily_returns.rename(columns={"timestamp": "Date"})
etf_daily_returns.set_index("Date")

,VOO,VGT,SPY
Date,,,
2020-08-10 04:00:00+00:00,0.003188,-0.003166,0.002959
2020-08-11 04:00:00+00:00,-0.008335,-0.017702,-0.008165
2020-08-12 04:00:00+00:00,0.013703,0.021928,0.013491
2020-08-13 04:00:00+00:00,-0.001452,0.002439,-0.001482
2020-08-14 04:00:00+00:00,-0.000032,-0.002466,0.000119
...,...,...,...
2022-08-01 04:00:00+00:00,-0.003010,-0.001783,-0.002961
2022-08-02 04:00:00+00:00,-0.006461,-0.005169,-0.006597
2022-08-03 04:00:00+00:00,0.015618,0.028997,0.015659


In [21]:
fig = px.line(etf_daily_returns, x="Date", y="VOO")
# generate scatter plot
fig.add_scatter(x=etf_daily_returns['Date'], y=etf_daily_returns['VGT'])
fig.add_scatter(x=etf_daily_returns['Date'], y=etf_daily_returns['SPY'])
fig.show()

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
tech_daily_returns = tech_daily_returns.reset_index()
tech_daily_returns = tech_daily_returns.rename(columns={"timestamp": "Date"})
tech_daily_returns.set_index("Date")

,GOOG,SPY,AMZN,NFLX,FB
Date,,,,,
2020-08-10 04:00:00+00:00,0.001077,0.002959,-0.006093,-0.022942,-0.020265
2020-08-11 04:00:00+00:00,-0.010547,-0.008165,-0.021438,-0.034031,-0.026122
2020-08-12 04:00:00+00:00,0.017766,0.013491,0.026478,0.018290,0.014680
2020-08-13 04:00:00+00:00,0.007852,-0.001482,-0.000386,0.012325,0.005425
2020-08-14 04:00:00+00:00,-0.007060,0.000119,-0.004113,0.002805,-0.000230
...,...,...,...,...,...
2022-08-31 04:00:00+00:00,-0.006915,-0.007609,-0.015226,0.013188,0.000000
2022-09-01 04:00:00+00:00,0.012826,0.003138,0.008283,0.028986,0.000000
2022-09-02 04:00:00+00:00,-0.016915,-0.010544,-0.002425,-0.017084,0.000000


In [ ]:
fig = px.line(tech_daily_returns, x="Date", y="GOOG")
# generate scatter plot
fig.add_scatter(x=tech_daily_returns['Date'], y=tech_daily_returns['SPY'])
fig.add_scatter(x=tech_daily_returns['Date'], y=tech_daily_returns['FB'])
fig.add_scatter(x=tech_daily_returns['Date'], y=tech_daily_returns['NFLX'])
fig.add_scatter(x=tech_daily_returns['Date'], y=tech_daily_returns['AMZN'])
fig.show()

In [ ]:
tech_cumulative_returns.hvplot(kind = "line", title="Tech Funds portfolios and the S&P 500 Cumulative Returns")

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

**Question** Based on the cumulative return data and the visualization, do any of the four fund portfolios outperform the S&P 500 Index?

#### Analyze Volatility 

In [ ]:
tech_daily_returns.hvplot(kind='box', title = "Tech Stocks Volatility")


:BoxWhisker   [Variable]   (value)

In [ ]:
drop_sp = tech_daily_returns.drop(columns=["SPY"]).copy()
tech_volatility = drop_sp.hvplot(kind='box', title='Volatility of The Four  stock Portfolios')

**Question** Based on the box plot visualization of just the four fund portfolios, which fund was the most volatile (with the greatest spread) and which was the least volatile (with the smallest spread)?

#### Analyze Risk

In [ ]:
tech_standards_deviation = tech_daily_returns.std()
tech_standards_deviation.sort_values

<bound method Series.sort_values of Date    219 days 10:25:37.692694312
GOOG                       0.045703
SPY                        0.011364
AMZN                       0.047524
NFLX                       0.032476
FB                         0.025604
dtype: object>

In [ ]:
# The annualized standard deviation (252 trading days) of the 4 portfolios and the S&P 500
tech_annualized_standards_deviation = tech_standards_deviation * np.sqrt(252)
tech_annualized_standards_deviation.sort_values

<bound method Series.sort_values of Date    3483 days 09:56:20.442600448
GOOG                        0.725515
SPY                         0.180398
AMZN                        0.754423
NFLX                        0.515535
FB                          0.406445
dtype: object>

In [ ]:
etf_standards_deviation = etf_daily_returns.std()
etf_standards_deviation

Date    210 days 05:09:38.157839864
VOO                        0.011244
VGT                        0.016423
SPY                        0.011305
dtype: object

In [ ]:
# The annualized standard deviation (252 trading days) of the 4 portfolios and the S&P 500
etf_annualized_standards_deviation = etf_standards_deviation * np.sqrt(252)
etf_annualized_standards_deviation

Date    3337 days 01:26:29.840507520
VOO                         0.178495
VGT                         0.260703
SPY                         0.179454
dtype: object

In [ ]:
# The rolling standard deviation of the 4 portfolios and the S&P 500 plot
tech_daily_returns.rolling(window=21).std().hvplot(kind="line",title="21-Day Rolling std of Tech Stock Portfolios and S&P 500")
etf_daily_returns.rolling(window=21).std().hvplot(kind="line",title="21-Day Rolling std of ETF Portfolios and S&P 500")

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [ ]:
tech_risk = drop_sp.rolling(window=21).std().hvplot(kind='line', title="21-Day Rolling Tech Stock Portfolios")

Based on the annualized standard deviation, which portfolios pose more risk than the S&P 500?

2. Based on the rolling metrics, does the risk of each portfolio increase at the same time that the risk of the S&P 500 increases?

3. Based on the rolling standard deviations of only the four fund portfolios, which portfolio poses the most risk? Does this change over time?

#### Analyze Risk-Return Profile 

In [ ]:
# The annual average return data for the tech fund portfolios and the S&P 500
trading_days = 252
tech_annual_average_returns = tech_daily_returns.mean() * trading_days
tech_annual_average_returns.sort_values

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.



<bound method Series.sort_values of GOOG   -0.212045
SPY     0.099532
AMZN   -0.494539
NFLX   -0.227244
FB     -0.064216
dtype: float64>

In [ ]:
# The annual average return data for the ETF fund portfolios and the S&P 500
trading_days = 252
etf_annual_average_returns = etf_daily_returns.mean() * trading_days
etf_annual_average_returns.sort_values

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.



<bound method Series.sort_values of VOO    0.122445
VGT    0.145946
SPY    0.122435
dtype: float64>

In [ ]:
# The annualized Sharpe Ratios for each of the 4 tech stock portfolios and the S&P 500.
tech_annualized_sharpe_ratio = tech_annual_average_returns/ tech_annualized_standards_deviation
tech_annualized_sharpe_ratio.sort_values()

AMZN   -0.655519
NFLX   -0.440792
GOOG   -0.292269
FB     -0.157994
SPY     0.551737
Date         NaN
dtype: object

In [ ]:
# The Sharpe ratios visualization as a bar chart
tech_annualized_sharpe_ratio.hvplot(kind="bar")

:Bars   [index]   (0)

In [ ]:
etf_annualized_sharpe_ratio = etf_annual_average_returns/ etf_annualized_standards_deviation
etf_annualized_sharpe_ratio.sort_values()

VGT     0.559817
SPY     0.682267
VOO     0.685986
Date         NaN
dtype: object

In [ ]:
# the Sharpe ratios visualization as a bar chart
etf_annualized_sharpe_ratio.hvplot(kind="bar")

:Bars   [index]   (0)

**Question** Which of the four portfolios offers the best risk-return profile? Which offers the worst?

#### Diversify Portfolio

In [ ]:
# The variance of the S&P 500 using a rolling 60-day window.
tech_sp_variance = tech_daily_returns["SPY"].rolling(window=60).var()
tech_sp_variance.tail()

519    0.000222
520    0.000220
521    0.000220
522    0.000211
523    0.000202
Name: SPY, dtype: float64

In [ ]:
# The covariance using a 60-day rolling window 
tech_covariance= tech_daily_returns['FB'].rolling(window=60).cov(tech_daily_returns["SPY"])
tech_covariance.tail()

519    3.591887e-07
520   -8.737662e-07
521    0.000000e+00
522    0.000000e+00
523    0.000000e+00
dtype: float64

In [ ]:
etf_sp_variance = etf_daily_returns['SPY'].rolling(window=60).var()
etf_sp_variance.tail()


497    0.000292
498    0.000272
499    0.000276
500    0.000258
501    0.000258
Name: SPY, dtype: float64

In [ ]:
# The covariance using a 60-day rolling window 
etf_covariance= etf_daily_returns['VGT'].rolling(window=60).cov(etf_daily_returns["SPY"])
etf_covariance.tail()

497    0.000385
498    0.000355
499    0.000362
500    0.000337
501    0.000337
dtype: float64

In [ ]:
# Tech beta based on the 60-day rolling covariance compared to the market (S&P 500)
tech_beta = (tech_covariance/tech_sp_variance) 
tech_beta.tail()

519    0.001619
520   -0.003966
521    0.000000
522    0.000000
523    0.000000
dtype: float64

In [ ]:
# The average of the 60-day rolling beta
tech_beta.mean()

1.3774060778119126

In [ ]:
# ETF beta based on the 60-day rolling covariance compared to the market (S&P 500)
etf_beta = (etf_covariance/etf_sp_variance) 
etf_beta.tail()

497    1.317745
498    1.306486
499    1.312849
500    1.308992
501    1.307866
dtype: float64

In [ ]:
# The average of the 60-day rolling beta
etf_beta.mean()

1.3003834419253426

In [ ]:
tech_beta = tech_beta.hvplot(kind='line',title="60-Day Rolling Beta: FB")
tech_beta 

:Curve   [index]   (0)

In [ ]:
etf_beta= etf_beta.hvplot(kind='line',title="60-Day Rolling Beta: VGT")
etf_beta


:Curve   [index]   (0)